- reference: https://www.learncodewithmike.com/2021/08/python-scraper-read-sqlite-database.html

## 爬取臺灣證交所的個股日成交資訊
1. 分析網頁結構
2. 開發Python網頁爬蟲
3. 建置SQLite資料庫
4. Python網頁爬蟲讀取資料庫

In [ ]:
from datetime import datetime
import requests
import sqlite3

response = requests.get(f'https://www.twse.com.tw/rwd/zh/afterTrading/STOCK_DAY?date=20230920&stockNo=2330')
response_data = response.json()['data']

print(response_data)

[['112/09/01', '15,194,921', '8,331,995,536', '543.00', '553.00', '543.00', '548.00', '-1.00', '12,739'], ['112/09/04', '9,772,925', '5,412,329,762', '549.00', '557.00', '549.00', '557.00', '+9.00', '12,158'], ['112/09/05', '14,337,041', '7,911,531,475', '553.00', '555.00', '550.00', '552.00', '-5.00', '13,714'], ['112/09/06', '14,442,757', '7,972,706,415', '556.00', '556.00', '550.00', '550.00', '-2.00', '12,752'], ['112/09/07', '22,610,385', '12,305,542,681', '546.00', '548.00', '542.00', '542.00', '-8.00', '41,564'], ['112/09/08', '16,702,110', '8,980,038,278', '535.00', '540.00', '535.00', '539.00', '-3.00', '36,672'], ['112/09/11', '15,540,773', '8,348,107,210', '539.00', '540.00', '536.00', '536.00', '-3.00', '31,155'], ['112/09/12', '17,135,765', '9,268,673,682', '536.00', '545.00', '536.00', '544.00', '+8.00', '20,545'], ['112/09/13', '16,836,487', '9,146,776,791', '545.00', '548.00', '541.00', '541.00', '-3.00', '18,111'], ['112/09/14', '18,377,284', '10,058,129,685', '544.00'

In [ ]:
#取得當日成交資訊: 透過迴圈讀取，並判斷
result = [data for index, data in enumerate(response_data) if '112/09/20' in response_data[index]]
result

[['112/09/20',
  '28,098,535',
  '15,065,407,249',
  '536.00',
  '538.00',
  '535.00',
  '535.00',
  '-3.00',
  '50,355']]

- 取得當日成交資訊: 使用datetime模組來取得當下的日期與指定格式
- 將請求網址date日期參數、與民國日期改為使用變數的方式帶入

In [ ]:
today = datetime.now().strftime('%Y%m%d')  #西元年(yyyymmdd)
chinese_today = f"{(datetime.now().year - 1911)}/{datetime.now().strftime('%m/%d')}"  #民國年(yyy/mm/dd)

response = requests.get(
    f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=date={today}&stockNo=2330')
response_data = response.json()['data']

result = [data for index, data in enumerate(response_data) if chinese_today in response_data[index]]

print(result)

[['112/09/25', '17,673,374', '9,285,699,418', '522.00', '529.00', '522.00', '525.00', '+3.00', '23,431']]


- 為了方便後續的資料識別，所以當日如果有成交資料的話，則在串列(List)中的第一個位置增加股票代碼：

In [ ]:
today = datetime.now().strftime('%Y%m%d')  #西元年(yyyymmdd)
chinese_today = f"{(datetime.now().year - 1911)}/{datetime.now().strftime('%m/%d')}"  #民國年(yyy/mm/dd)

response = requests.get(
    f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=date={today}&stockNo=2330')
response_data = response.json()['data']

result = [data for index, data in enumerate(response_data) if chinese_today in response_data[index]]

if result:  #如果有資料
    result[0].insert(0, '2330')

print(result)

[['2330', '112/09/25', '17,673,374', '9,285,699,418', '522.00', '529.00', '522.00', '525.00', '+3.00', '23,431']]


###  **建置SQLite資料庫，Python網頁爬蟲讀取資料庫**

- downlaod "DB Browser for SQLite"
- SQLite資料庫建置完成後，開啟app.py檔案，利用sqlite3模組連接與設定撈取StockNumbers資料表中的StockNo欄位資料SQL指令

In [ ]:
from datetime import datetime
import requests
import sqlite3

today = datetime.now().strftime('%Y%m%d')  #西元年(yyyymmdd)
chinese_today = f"{(datetime.now().year - 1911)}/{datetime.now().strftime('%m/%d')}"  #民國年(yyy/mm/dd)

conn = sqlite3.connect('Stock.db')
cursor = conn.cursor()
cursor.execute('SELECT StockNo FROM StockNumbers')

response = requests.get(
    f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={today}&stockNo=2330')
response_data = response.json()['data']

result = [data for index, data in enumerate(response_data) if chinese_today in response_data[index]]

if result:  #如果有資料
    result[0].insert(0, '2330')

print(result)

[['2330', '112/09/25', '17,673,374', '9,285,699,418', '522.00', '529.00', '522.00', '525.00', '+3.00', '23,431']]


- 呼叫sqlite3模組的 fetchall( ) 方法，執行撈取StockNo欄位的所有股票代碼資料SQL指令
- 並透過迴圈來進行讀取與替換既有的股票代碼

In [ ]:
conn = sqlite3.connect('Stock.db')
cursor = conn.cursor()
cursor.execute('SELECT StockNo FROM StockNumbers')

for stock_no in cursor.fetchall():
    response = requests.get(f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={today}&stockNo={stock_no[0]}')
    response_data = response.json()['data']

    result = [data for index, data in enumerate(response_data) if chinese_today in response_data[index]]

    if result:  #如果有資料
	    result[0].insert(0, stock_no[0])

print(result)

[[2409, '112/09/25', '11,874,838', '198,222,528', '16.70', '16.80', '16.60', '16.70', '+0.05', '3,403']]


- 最後，為了讓三個股票代碼資料能夠存放在同一個串列(List)中，所以另外定義了combined串列(List)，加入Python網頁爬蟲爬取的所有股票代碼資料

In [ ]:
from datetime import datetime
import requests
import sqlite3


today = datetime.now().strftime('%Y%m%d')  #西元年(yyyymmdd)
chinese_today = f"{(datetime.now().year - 1911)}/{datetime.now().strftime('%m/%d')}"  #民國年(yyy/mm/dd)

conn = sqlite3.connect('Stock.db')
cursor = conn.cursor()
cursor.execute('SELECT StockNo FROM StockNumbers')

combined = []  #合併結果
for stock_no in cursor.fetchall():
    response = requests.get(f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={today}&stockNo={stock_no[0]}')
    response_data = response.json()['data']

    result = [data for index, data in enumerate(response_data) if chinese_today in response_data[index]]

    if result:  #如果有資料
      result[0].insert(0, stock_no[0])
      combined.append(result[0])

print(combined)

[[2330, '112/09/25', '17,673,374', '9,285,699,418', '522.00', '529.00', '522.00', '525.00', '+3.00', '23,431'], [2382, '112/09/25', '38,984,310', '8,811,557,891', '227.50', '228.00', '222.50', '226.00', '-1.50', '28,692'], [2409, '112/09/25', '11,874,838', '198,222,528', '16.70', '16.80', '16.60', '16.70', '+0.05', '3,403']]
